In [1]:
from PExp import *

In [2]:
exp1 = PExp(6,population=['b','b','a','c','a','b','b','c'],prior_prob=[1,1],n_trials=50000)
exp2 = PExp(6,population={'b':4,'c':2,'a':2},prior_prob=[1,1])
exp3 = PExp(population=[('a','R'),('a','R'),('b','B'),('b','R')])
exp4 = PExp(6,population=[('b',),('b',),('a',),('c',),('a',),('b',),('b',),('c',)],prior_prob=[1,1])
exp5 = PExp(prior_prob=[1,2,1])
exp6 = PExp(prior_prob=[1,2,1],alphabet=['a','b','c'])
exp7 = PExp(prior_prob=[1,2,1],alphabet=[('a','0'),('b','1'),('c','2')])
exp8 = PExp(alphabet=['a','b','c'])
exp9 = PExp(4)
#exp1.info()
#exp2.info()
#exp3.info()
#exp4.info()
#exp5.info()
#exp6.info()
#exp7.info()
#exp8.info()
#exp9.info()
#exp1.info()
#exp1.make_exp()
#exp1.info()

exp1 = PExp(6)
exp2 = PExp(alphabet=['T','C'])
exp1.info()
exp2.info()
exp3 = exp1*exp2
exp3.info()
exp3.make_exp(10000)
exp3.info()

----------------------------------------------------------------------------------------------------
Population: None
Alphabet: [('1',), ('2',), ('3',), ('4',), ('5',), ('6',)]
Prior_prob: [ 0.16666667  0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]
N_outcomes: 6
N_trials: 6000
Post_prob: [ 0.  0.  0.  0.  0.  0.]
Exp_dict:
{('1',): {'post_prob': 0.0, 'prior_prob': 0.16666666666666666},
 ('2',): {'post_prob': 0.0, 'prior_prob': 0.16666666666666666},
 ('3',): {'post_prob': 0.0, 'prior_prob': 0.16666666666666666},
 ('4',): {'post_prob': 0.0, 'prior_prob': 0.16666666666666666},
 ('5',): {'post_prob': 0.0, 'prior_prob': 0.16666666666666666},
 ('6',): {'post_prob': 0.0, 'prior_prob': 0.16666666666666666}}

----------------------------------------------------------------------------------------------------
Population: None
Alphabet: [('T',), ('C',)]
Prior_prob: [ 0.5  0.5]
N_outcomes: 2
N_trials: 2000
Post_prob: [ 0.  0.]
Exp_dict:
{('C',): {'post_prob': 0.0, 'prior_prob': 0.5},

In [3]:
exp_carte = PExp(alphabet=['A','2','3','4','5','6','7','8','9','J','Q','K'])
exp_semi = PExp(alphabet=['C','Q','F','P'])
exp_cartefrancesi = exp_carte * exp_semi
exp_cartefrancesi.info()

----------------------------------------------------------------------------------------------------
Population: None
Alphabet: [('A', 'C'), ('A', 'Q'), ('A', 'F'), ('A', 'P'), ('2', 'C'), ('2', 'Q'), ('2', 'F'), ('2', 'P'), ('3', 'C'), ('3', 'Q'), ('3', 'F'), ('3', 'P'), ('4', 'C'), ('4', 'Q'), ('4', 'F'), ('4', 'P'), ('5', 'C'), ('5', 'Q'), ('5', 'F'), ('5', 'P'), ('6', 'C'), ('6', 'Q'), ('6', 'F'), ('6', 'P'), ('7', 'C'), ('7', 'Q'), ('7', 'F'), ('7', 'P'), ('8', 'C'), ('8', 'Q'), ('8', 'F'), ('8', 'P'), ('9', 'C'), ('9', 'Q'), ('9', 'F'), ('9', 'P'), ('J', 'C'), ('J', 'Q'), ('J', 'F'), ('J', 'P'), ('Q', 'C'), ('Q', 'Q'), ('Q', 'F'), ('Q', 'P'), ('K', 'C'), ('K', 'Q'), ('K', 'F'), ('K', 'P')]
Prior_prob: [ 0.02083333  0.02083333  0.02083333  0.02083333  0.02083333  0.02083333
  0.02083333  0.02083333  0.02083333  0.02083333  0.02083333  0.02083333
  0.02083333  0.02083333  0.02083333  0.02083333  0.02083333  0.02083333
  0.02083333  0.02083333  0.02083333  0.02083333  0.02083333  0.

In [4]:
e1 = Event([('K','C'),('6','F')],exp_cartefrancesi)
print(e1.get_prob())
print(e1)
Fraction(e1.get_prob()).limit_denominator(100)

0.0416666666667
[('K', 'C'), ('6', 'F')]: 0.0416666666667


Fraction(1, 24)

In [5]:
e2 = Event([('*','C')],exp_cartefrancesi)
print(e2.get_prob())
print(e2)
Fraction(e2.get_prob()).limit_denominator(100)

0.25
[('A', 'C'), ('2', 'C'), ('3', 'C'), ('4', 'C'), ('5', 'C'), ('6', 'C'), ('7', 'C'), ('8', 'C'), ('9', 'C'), ('J', 'C'), ('Q', 'C'), ('K', 'C')]: 0.25


Fraction(1, 4)

In [6]:
e3 = Event([('J','*')],exp_cartefrancesi)
print(e3.get_prob())
print(e3)
Fraction(e3.get_prob()).limit_denominator(100)

0.0833333333333
[('J', 'C'), ('J', 'Q'), ('J', 'F'), ('J', 'P')]: 0.0833333333333


Fraction(1, 12)

In [7]:
print((-e3).get_prob() + e3.get_prob())
print(e3 + (-e3))

1.0
[('2', 'C'), ('2', 'F'), ('2', 'P'), ('2', 'Q'), ('3', 'C'), ('3', 'F'), ('3', 'P'), ('3', 'Q'), ('4', 'C'), ('4', 'F'), ('4', 'P'), ('4', 'Q'), ('5', 'C'), ('5', 'F'), ('5', 'P'), ('5', 'Q'), ('6', 'C'), ('6', 'F'), ('6', 'P'), ('6', 'Q'), ('7', 'C'), ('7', 'F'), ('7', 'P'), ('7', 'Q'), ('8', 'C'), ('8', 'F'), ('8', 'P'), ('8', 'Q'), ('9', 'C'), ('9', 'F'), ('9', 'P'), ('9', 'Q'), ('A', 'C'), ('A', 'F'), ('A', 'P'), ('A', 'Q'), ('J', 'C'), ('J', 'F'), ('J', 'P'), ('J', 'Q'), ('K', 'C'), ('K', 'F'), ('K', 'P'), ('K', 'Q'), ('Q', 'C'), ('Q', 'F'), ('Q', 'P'), ('Q', 'Q')]: 1.0


In [8]:
e3|e2

[('J', 'C')]: 0.0833333333333

In [9]:
Fraction(0.0833333333333).limit_denominator(100)

Fraction(1, 12)

In [10]:
exptemp = PExp(alphabet=e2.abc)
exptemp.info()
evtemp = Event(e3.abc,exptemp)
print(e3.abc)
evtemp.abc
#evtemp.prob

----------------------------------------------------------------------------------------------------
Population: None
Alphabet: [('A', 'C'), ('2', 'C'), ('3', 'C'), ('4', 'C'), ('5', 'C'), ('6', 'C'), ('7', 'C'), ('8', 'C'), ('9', 'C'), ('J', 'C'), ('Q', 'C'), ('K', 'C')]
Prior_prob: [ 0.08333333  0.08333333  0.08333333  0.08333333  0.08333333  0.08333333
  0.08333333  0.08333333  0.08333333  0.08333333  0.08333333  0.08333333]
N_outcomes: 12
N_trials: 12000
Post_prob: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Exp_dict:
{('2', 'C'): {'post_prob': 0.0, 'prior_prob': 0.083333333333333329},
 ('3', 'C'): {'post_prob': 0.0, 'prior_prob': 0.083333333333333329},
 ('4', 'C'): {'post_prob': 0.0, 'prior_prob': 0.083333333333333329},
 ('5', 'C'): {'post_prob': 0.0, 'prior_prob': 0.083333333333333329},
 ('6', 'C'): {'post_prob': 0.0, 'prior_prob': 0.083333333333333329},
 ('7', 'C'): {'post_prob': 0.0, 'prior_prob': 0.083333333333333329},
 ('8', 'C'): {'post_prob': 0.0, 'prior_prob': 0.0833

[('J', 'C')]